In [4]:
#    Air stagnation indices over the Euro-Mediterranean region
#    Copyright (C) 2020  Carlos Ordóñez García
#    Copyright (C) 2020  José Manuel Garrido Pérez
#    Copyright (C) 2020  Javier Ruano Ruano
#
#    This program is free software: you can redistribute it and/or modify
#    it under the terms of the GNU General Public License as published by
#    the Free Software Foundation, either version 3 of the License, or
#    (at your option) any later version.
#
#    This program is distributed in the hope that it will be useful,
#    but WITHOUT ANY WARRANTY; without even the implied warranty of
#    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
#    GNU General Public License for more details.
#
#    You should have received a copy of the GNU General Public License
#    along with this program.  If not, see <https://www.gnu.org/licenses/>.


#   This air stagnation index (ASI) was introduced by Wang et al. (2016, 2018), using meteorological and air quality data over 
# China, Europe and the United States. It looks for meteorological conditions conducive to elevated concentrations of PM2.5 
# (particulate matter with aerodynamic diameter ≤ 2.5 µm). For that purpose, it uses near-surface wind speed (W), 
# atmospheric boundary layer height (BLH) and precipitation as indices of atmospheric horizontal dispersion capability, 
# vertical mixing and wet deposition, respectively.

#   The first two meteorological variables are used to fit equations that identify the conditions with above-average daily 
# PM2.5 concentrations for each season:

# Seasons changes
# Spring: BLH = 3.57 * 103 * exp(-3.35 * W) + 0.352
# Summer: BLH = 7.66 * 10 * exp(-2.12 * W) + 0.443
# Autumn: BLH = 1.88 * 104 * exp(-5.15 * W) + 0.440
# Winter: BLH = 0.759 * exp(-0.6 * W) + 0.264

#Here, Wang's ASI has been computed over land areas by using meteorological fields from the ERA5 reanalysis. A grid cell is considered as stagnant on a given day if two conditions are simultaneously met:
#BLH at 12UTC is below that computed from the equations above using the daily average wind speed at 10 m, and the daily accumulated precipitation is below 1 mm.

# Wang, X., Dickinson, R. E., Su, L., Zhou, C., Wang, K., 2018. PM2.5 pollution in China and how it has been exacerbated by terrain and meteorological conditions. B. Am. Meteorol. Soc. 99(1), 105-119. https://doi.org/10.1175/BAMS-D-16-0301.1.

# Wang, X., Wang, K., Su, L., 2016. Contribution of atmospheric diffusion conditions to the recent improvement in air quality in China. Sci. Rep. 6, 36404. https://doi.org/10.1038/srep36404.

# Website : http://steady-ucm.org
# https://github.com/JavierRuano/ASI_Steady

import xarray as xr
import numpy as np
import itertools
import pandas as pd

Boundary_Layer_Hei=xr.open_mfdataset('Dataset/*boundary_layer_height*.nc',parallel=True,combine='nested', concat_dim='time')
Surface_Wind_Speed_u10=xr.open_mfdataset('Dataset/*10m_u_component_of_wind*.nc',parallel=True,combine='nested', concat_dim='time')
Surface_Wind_Speed_v10=xr.open_mfdataset('Dataset/*10m_v_component_of_wind*.nc',parallel=True,combine='nested', concat_dim='time')
total_precipitation=xr.open_mfdataset('Dataset/*total_precipitation*.nc',parallel=True,combine='nested', concat_dim='time')

Surface_Wind_Speed=Surface_Wind_Speed_u10.merge(Surface_Wind_Speed_v10)

Surface_Wind_Mod=xr.ufuncs.sqrt(Surface_Wind_Speed.u10**2 + Surface_Wind_Speed.v10**2).sel(time=slice(Surface_Wind_Speed['time'][0].values,Surface_Wind_Speed['time'][-3].values))

List1=np.arange(len(Surface_Wind_Speed.time.values))[:int((len(Surface_Wind_Speed.time.values)/2)-1)]
List2=np.arange(len(Surface_Wind_Speed.time.values))[:int((len(Surface_Wind_Speed.time.values)/2)-1)]+4

time_Index=Surface_Wind_Speed.time.values[::4]
latitude_Index=Surface_Wind_Speed.latitude.values
longitude_Index=Surface_Wind_Speed.longitude.values
Module_Wind_Surface=np.add.reduceat(Surface_Wind_Mod.values,[val for pair in zip(List1, List2) for val in pair])[::4]/4
Sum_Rain=np.add.reduceat(total_precipitation.tp.values,[val for pair in zip(List1, List2) for val in pair])[::4]
Wind_Surface_Array=xr.DataArray(Module_Wind_Surface,coords=[time_Index,latitude_Index,longitude_Index],dims=['time','latitude','longitude'])
Rain_Array=xr.DataArray(Sum_Rain,coords=[time_Index,latitude_Index,longitude_Index],dims=['time','latitude','longitude'])
Index=pd.DataFrame(Wind_Surface_Array.time.values,columns=['date'])

def define_period(season):
    if(season in [3,4,5]):
        return 1
    elif(season in [6,7,8]):
        return 2
    elif(season in [9,10,11]):
        return 3
    else:
        return 0
count=0
operationSeason=Index['date'].astype('datetime64').dt.month.apply(define_period).values


def s1(b):
    return np.multiply(np.multiply(3.57,1000),np.exp(-3.35*b))+0.352
def s2(b):
    return np.multiply(np.multiply(7.66,10),np.exp(-2.12*b))+0.443
def s3(b):
    return np.multiply(np.multiply(1.88,10000),np.exp(-5.15*b))+0.440
def s4(b):
    return np.multiply(0.759,np.exp(-0.6*b))+0.264
function = {
        "1": s1,
        "2": s2,
        "3": s3,
        "0": s4
    }

 


count=0
for i in operationSeason:
    Wind_Surface_Array[count]=function[str(i)](Wind_Surface_Array[count])
    count+=1


(((2*(Rain_Array>0.001)) -(Boundary_Layer_Hei['blh'].compute()<Wind_Surface_Array)*1)>=1).to_dataset(name='ASI_WANG')

<xarray.Dataset>
Dimensions:    (latitude: 68, longitude: 81, time: 14975)
Coordinates:
  * time       (time) datetime64[ns] 1979-01-01 1979-01-02 ... 2019-12-31
  * latitude   (latitude) float32 75.0 74.25 73.5 72.75 ... 26.25 25.5 24.75
  * longitude  (longitude) float32 -20.0 -19.25 -18.5 -17.75 ... 38.5 39.25 40.0
Data variables:
    ASI_WANG   (time, latitude, longitude) bool False False ... False False